<a href="https://colab.research.google.com/github/gcasaldi/football/blob/main/daily_predictions_fd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🌍 Football Predictions — Full Competitions Browser (Football-Data.org)
**Novità:**  
- Elenco **tutte le competizioni** disponibili con la tua key.  
- Ricerca delle **partite di oggi** su **tutte** le competizioni (no filtro per lega).  
- Filtri rapidi per competizione e per **team name** (es. *Italy*).  
- Predizioni con **Poisson** per 1X2, Doppia Chance, U/O 1.5 & 2.5, GG/NG, e combo `1+Over 1.5 / 2.5 / Under 2.5`.  


## 0) Setup

In [1]:

# Se sei su Colab, sblocca l'installazione:
# !pip -q install requests pandas numpy gradio

import os, math, json, datetime as dt, numpy as np, pandas as pd, requests
import gradio as gr

API_KEY = "0f88fed651334f7cacb2fcb856541698"  # tua chiave
BASE_URL = "https://api.football-data.org/v4"
HEADERS = {"X-Auth-Token": API_KEY}

MAX_GOALS = 10
LOOKBACK_MATCHES = 10


## 1) API Helpers & Lista Competizioni

In [2]:

def fd_get(path, params=None):
    url = f"{BASE_URL}/{path.lstrip('/')}"
    r = requests.get(url, headers=HEADERS, params=params, timeout=30)
    if r.status_code == 429:
        raise RuntimeError("Rate limit raggiunto. Attendi e riprova.")
    r.raise_for_status()
    return r.json()

def list_competitions():
    data = fd_get("competitions")
    comps = pd.json_normalize(data.get("competitions", []))
    cols = ["id","name","code","type","area.name","plan"]
    cols = [c for c in cols if c in comps.columns]
    return comps[cols].sort_values("name")

comps_df = list_competitions()
comps_df.head(20)


,id,name,code,type,area.name,plan
6,2002,Bundesliga,BL1,LEAGUE,Germany,TIER_ONE
0,2013,Campeonato Brasileiro Série A,BSA,LEAGUE,Brazil,TIER_ONE
1,2016,Championship,ELC,LEAGUE,England,TIER_ONE
10,2152,Copa Libertadores,CLI,CUP,South America,TIER_FOUR
8,2003,Eredivisie,DED,LEAGUE,Netherlands,TIER_ONE
4,2018,European Championship,EC,CUP,Europe,TIER_ONE
12,2000,FIFA World Cup,WC,CUP,World,TIER_ONE
5,2015,Ligue 1,FL1,LEAGUE,France,TIER_ONE
2,2021,Premier League,PL,LEAGUE,England,TIER_ONE
9,2017,Primeira Liga,PPL,LEAGUE,Portugal,TIER_ONE


## 2) Partite di **oggi** su tutte le competizioni

In [3]:

def matches_today_all():
    today = dt.date.today().isoformat()
    data = fd_get("matches", params={"dateFrom": today, "dateTo": today})
    df = pd.json_normalize(data.get("matches", []))
    if df.empty:
        return df
    df["utcDate"] = pd.to_datetime(df["utcDate"], utc=True)
    show_cols = ["utcDate","competition.name","homeTeam.name","awayTeam.name","status"]
    show_cols = [c for c in show_cols if c in df.columns]
    return df.sort_values("utcDate")[show_cols]

all_today = matches_today_all()
all_today.head(30)


""


## 3) Forza squadre (rolling) e Poisson

In [4]:

def competition_matches(competition_id:int, season:int):
    data = fd_get(f"competitions/{competition_id}/matches", params={"season": season})
    return pd.json_normalize(data.get("matches", []))

def last_matches_for_team(team_id:int, seasons:list):
    frames=[]
    for comp in list_competitions().itertuples(index=False):
        for y in seasons:
            try:
                m = competition_matches(int(comp.id), int(y))
                if m.empty:
                    continue
                frames.append(m)
            except Exception:
                continue
    df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    if df.empty: return df
    sel = df[(df["homeTeam.id"]==team_id)|(df["awayTeam.id"]==team_id)]
    sel = sel[sel["status"]=="FINISHED"].copy()
    sel["utcDate"] = pd.to_datetime(sel["utcDate"], utc=True)
    return sel.sort_values("utcDate", ascending=False)

def rolling_goals_rate(team_id:int, seasons:list, n:int=10):
    m = last_matches_for_team(team_id, seasons)
    if m.empty:
        return 1.4,1.4
    gf_list, ga_list = [], []
    for _, r in m.iterrows():
        hs = r.get("score.fullTime.home"); as_ = r.get("score.fullTime.away")
        if pd.isna(hs) or pd.isna(as_): continue
        if r["homeTeam.id"]==team_id:
            gf_list.append(hs); ga_list.append(as_)
        else:
            gf_list.append(as_); ga_list.append(hs)
    gf = np.mean(gf_list[:n]) if gf_list else 1.4
    ga = np.mean(ga_list[:n]) if ga_list else 1.4
    return float(gf), float(ga)

def estimate_lambdas(home_id:int, away_id:int, seasons:list, n:int=LOOKBACK_MATCHES):
    hg_for, hg_against = rolling_goals_rate(home_id, seasons, n)
    ag_for, ag_against = rolling_goals_rate(away_id, seasons, n)
    lam_h = max(0.05, (hg_for + ag_against)/2)
    lam_a = max(0.05, (ag_for + hg_against)/2)
    return lam_h, lam_a

def pois_matrix(lh: float, la: float, max_goals:int=MAX_GOALS):
    pmf_h = np.array([np.exp(-lh)*(lh**i)/math.factorial(i) for i in range(max_goals+1)])
    pmf_a = np.array([np.exp(-la)*(la**j)/math.factorial(j) for j in range(max_goals+1)])
    M = np.outer(pmf_h, pmf_a)
    return M / M.sum()

def market_probs(M: np.ndarray, line15: float=1.5, line25: float=2.5):
    max_g = M.shape[0]-1
    p1=pX=p2=0.0; u15=o15=u25=o25=0.0; gg=0.0
    c1o15=c1o25=c1u25=0.0
    for i in range(max_g+1):
        for j in range(max_g+1):
            p = float(M[i,j])
            if i>j: p1+=p
            elif i==j: pX+=p
            else: p2+=p
            if (i+j) > line15: o15+=p
            else: u15+=p
            if (i+j) > line25: o25+=p
            else: u25+=p
            if i>=1 and j>=1: gg+=p
            if i>j and (i+j) > 1.5: c1o15 += p
            if i>j and (i+j) > 2.5: c1o25 += p
            if i>j and (i+j) <= 2.5: c1u25 += p
    return {
        "1":p1,"X":pX,"2":p2,
        "DC_1X":p1+pX,"DC_X2":pX+p2,"DC_12":p1+p2,
        "U1.5":u15,"O1.5":o15,"U2.5":u25,"O2.5":o25,
        "GG":gg,"NG":1-gg,
        "1+O1.5":c1o15,"1+O2.5":c1o25,"1+U2.5":c1u25
    }


## 4) Predizioni **oggi** con filtri (competizione/team)

In [5]:

def predict_today_filtered(team_contains:str="", competition_name:str="", seasons=None, lookback:int=LOOKBACK_MATCHES):
    seasons = seasons or [dt.date.today().year, dt.date.today().year-1]
    raw = fd_get("matches", params={"dateFrom": dt.date.today().isoformat(),
                                    "dateTo": dt.date.today().isoformat()})
    df = pd.json_normalize(raw.get("matches", []))
    if df.empty:
        return pd.DataFrame()
    df["utcDate"] = pd.to_datetime(df["utcDate"], utc=True)
    df = df[df["status"].isin(["SCHEDULED","TIMED","POSTPONED"])]
    if competition_name:
        df = df[df["competition.name"].str.contains(competition_name, case=False, na=False)]
    if team_contains:
        mask = df["homeTeam.name"].str.contains(team_contains, case=False, na=False) |                df["awayTeam.name"].str.contains(team_contains, case=False, na=False)
        df = df[mask]
    rows=[]
    for _, r in df.iterrows():
        hid, aid = int(r["homeTeam.id"]), int(r["awayTeam.id"])
        home, away = r["homeTeam.name"], r["awayTeam.name"]
        try:
            lh, la = estimate_lambdas(hid, aid, seasons, n=lookback)
            M = pois_matrix(lh, la, MAX_GOALS)
            mp = market_probs(M)
            picks = {
                "1": mp["1"], "X": mp["X"], "2": mp["2"],
                "DC_1X": mp["DC_1X"], "DC_X2": mp["DC_X2"], "DC_12": mp["DC_12"],
                "O2.5": mp["O2.5"], "U2.5": mp["U2.5"],
                "GG": mp["GG"], "NG": mp["NG"],
                "1+O1.5": mp["1+O1.5"], "1+O2.5": mp["1+O2.5"], "1+U2.5": mp["1+U2.5"]
            }
            best = max(picks, key=picks.get)
            rows.append({
                "kickoff_utc": r["utcDate"],
                "competition": r["competition.name"],
                "home": home, "away": away,
                "P(1)": mp["1"], "P(X)": mp["X"], "P(2)": mp["2"],
                "P(DC_1X)": mp["DC_1X"], "P(DC_X2)": mp["DC_X2"], "P(DC_12)": mp["DC_12"],
                "P(U1.5)": mp["U1.5"], "P(O1.5)": mp["O1.5"],
                "P(U2.5)": mp["U2.5"], "P(O2.5)": mp["O2.5"],
                "P(GG)": mp["GG"], "P(NG)": 1-mp["GG"],
                "P(1+O1.5)": mp["1+O1.5"], "P(1+O2.5)": mp["1+O2.5"], "P(1+U2.5)": mp["1+U2.5"],
                "Pick": best
            })
        except Exception as e:
            rows.append({
                "kickoff_utc": r["utcDate"],
                "competition": r["competition.name"],
                "home": home, "away": away,
                "Pick": f"errore: {e}"
            })
    out = pd.DataFrame(rows).sort_values("kickoff_utc")
    return out

# Esempio: cerca tutte le partite che coinvolgono "Italy" oggi (se ce ne sono)
pred_italy_today = predict_today_filtered(team_contains="Italy")
pred_italy_today.head()


""


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

## 5) Export CSV (directory corrente — Colab)

In [6]:

def export_csv(df: pd.DataFrame, filename: str = "predizioni_oggi.csv"):
    if df is None or df.empty:
        print("Nessun dato da salvare.")
        return None
    df.to_csv(filename, index=False)
    print("Salvato:", filename)
    return filename


## 6) Interfaccia Gradio

In [7]:

def ui_run(team_contains, competition_name, lookback, seasons_text):
    seasons = []
    if seasons_text.strip():
        try:
            seasons = [int(s.strip()) for s in seasons_text.split(",")]
        except:
            seasons = [dt.date.today().year, dt.date.today().year-1]
    else:
        seasons = [dt.date.today().year, dt.date.today().year-1]
    df = predict_today_filtered(team_contains, competition_name, seasons, int(lookback))
    if df.empty:
        return "Nessuna partita trovata per i filtri scelti.", ""
    show = df.copy()
    for col in show.columns:
        if col.startswith("P("):
            show[col] = (show[col]*100).round(1).astype(str) + "%"
    return "", show.to_markdown(index=False)

with gr.Blocks() as demo:
    gr.Markdown("### Partite di oggi — browser completo")
    with gr.Row():
        team = gr.Textbox(label="Filtra per Team (es. Italy)", value="")
        comp = gr.Textbox(label="Filtra per Competizione (es. Nations League)", value="")
    with gr.Row():
        lookback = gr.Slider(5, 20, value=LOOKBACK_MATCHES, step=1, label="Match storici per forza")
        seasons = gr.Textbox(label="Stagioni (comma-separate)", value=f"{dt.date.today().year},{dt.date.today().year-1}")
    btn = gr.Button("Calcola")
    warn = gr.Markdown()
    table = gr.Markdown()
    btn.click(ui_run, [team, comp, lookback, seasons], [warn, table])

# Per avviare in Colab: demo.launch(share=True)



## 7) Come usarlo
1. Esegui le celle 0–1 per collegarti all'API e vedere le competizioni disponibili.  
2. Esegui la cella **2** per vedere tutte le partite di **oggi** a livello globale.  
3. Usa la funzione `predict_today_filtered(team_contains="Italy")` per trovare i match dell'Italia (se presenti).  
4. Avvia la **UI** con `demo.launch(share=True)`.  
5. Esporta con `export_csv(pred_df)`.  
